In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import  matplotlib.pyplot as plt 
import seaborn as sns
plt.style.use('fivethirtyeight')
col_pal = sns.color_palette()


In [ ]:
#load_each datasets

oil_df = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/oil.csv')
holidays_df = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv')
store_df = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/stores.csv')
trans_df = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/transactions.csv')
train = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/train.csv')
test = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/test.csv')

# **Let's Analyse oil data for now**

In [ ]:
print(oil_df.info())
oil_df.set_index('date')


# Let's observeb the Trend of data

In [ ]:
oil_df.plot(style='.',figsize =(15,5),color = col_pal[0],title ='dcoilwtico' )

In [ ]:
oil_df.index = pd.to_datetime(oil_df.index)

In [ ]:
oil_df = oil_df.set_index('date')


# see what is in the holidays data

In [ ]:
holidays_df[holidays_df['transferred']==False]
#nothing important

In [ ]:
trans_df.head()

In [ ]:
#trans_df = trans_df.set_index('date')
trans_df['transactions'].plot(style ='.',figsize =(15,5),color = col_pal[1],title='transactions')

In [ ]:
train.head()

# Let's visit Train dataset


In [ ]:
train.family.unique()

In [ ]:
auto = train[train['family']=='AUTOMOTIVE']
auto = train.set_index('date')
auto['sales'].plot(style ='.',figsize =(15,5),color = col_pal[3],title='automotive sales')

In [ ]:
np.random.randint(0,5)

In [ ]:
len(col_pal)

In [ ]:
for i in train.family.unique():
    data = train[train['family']==i]
    data = data.set_index('date')
    data['sales'].plot(style ='.',figsize =(15,5),color = col_pal[np.random.randint(0,5)],title=i+' sales')
    

In [ ]:
#load_each datasets

oil_df = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/oil.csv')
holidays_df = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv')
store_df = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/stores.csv')
trans_df = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/transactions.csv')
train = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/train.csv')
test = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/test.csv')

In [ ]:
train

In [ ]:
oil_df

In [ ]:
oil_df.rename(columns={'dcoilwtico':'oil_price'},inplace = True)
train = pd.merge(train,oil_df,on='date',how='left')
test = pd.merge(test,oil_df,on ='date',how ='left')

In [ ]:
train

In [ ]:
train = pd.merge(train,store_df,on='store_nbr',how='left')
test = pd.merge(test,store_df,on ='store_nbr',how ='left')

# holidays


In [ ]:
holidays_df.locale.unique()

In [ ]:
holidays_df.type.unique()

In [ ]:
holidays_df.head()

In [ ]:
condition = (holidays_df.transferred == True)|(holidays_df.type =='Work Day')
special_days = holidays_df.drop(holidays_df[condition].index)
special_days = special_days.drop(['type', 'description', 'transferred'], axis=1)

In [ ]:
#special_days = special_days.drop(['type', 'description', 'transferred'], axis=1)

In [ ]:
special_days['special days'] = 1
special_days = special_days.drop(special_days[special_days.date.duplicated()].index)


In [ ]:
national = special_days[special_days.locale == 'National']
regional = special_days[special_days.locale == 'Regional']
local = special_days[special_days.locale == 'Local']

In [ ]:
national = national.drop(['locale','locale_name'],axis =1)
train = pd.merge(train,national,on='date',how='left')
test = pd.merge(test,national,on='date',how ='left')

In [ ]:
for i, j in zip(regional.date, regional.locale_name):
    train['special days'][(train.date == i) & (train.state == j)] = 1

In [ ]:
for i, j in zip(regional.date, regional.locale_name):
    test['special days'][(test.date == i) & (test.state == j)] = 1

In [ ]:
for i, j in zip(local.date, local.locale_name):
    train['special days'][(train.date == i) & (train.city == j)] = 1

In [ ]:
for i, j in zip(local.date, local.locale_name):
    test['special days'][(test.date == i) & (test.city == j)] = 1

In [ ]:
train

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
train.oil_price = train.oil_price.fillna(method ='bfill')
test.oil_price = test.oil_price.fillna(method='bfill')

In [ ]:
train['special days']= train['special days'].fillna(0)
test['special days'] =test['special days'].fillna(0)

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

# Remove Unnessesary cols


In [ ]:
train = train.drop(['date','store_nbr','state','city','cluster'],axis =1)
test = test.drop(['date','store_nbr','state','city','cluster'],axis =1)

In [ ]:
train.head()
#test.head()

In [ ]:
test.head()

In [ ]:
train.set_index('id')
test.set_index('id')

In [ ]:
train = pd.get_dummies(train)
test = pd.get_dummies(test)

In [ ]:
train.corr()['sales'].sort_values(ascending= True).head(10)


In [ ]:
train.corr()['sales'].sort_values(ascending =False).head(10)

In [ ]:
X = train.drop(['sales'],axis =1)
y = train.sales

In [ ]:
X_test = test

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size =0.2,random_state= 42)


In [ ]:
from sklearn.linear_model import LinearRegression
model_lr = LinearRegression()
model_lr.fit(X_train,y_train)

In [ ]:
y_pred_lr = abs(model_lr.predict(X_val))

In [ ]:
y_pred_lr

In [ ]:
y_val

In [ ]:
X.shape

In [ ]:
y_pred = model_lr.predict(X_test)
output = pd.DataFrame({'id': X_test.id, 'sales': np.round(y_pred, 3)})
output.to_csv('submission.csv', index=False)

In [ ]:
def linear_reg(X,y,itr,lr):
    w_0 = np.random.randn(X.shape[1],y.shape[0])
    w= w_0
    past_cost =[]
    past_w =[w_0]
    for i in range(itr):
        y_pred = X @ w
        error = y_pred - y
        loss = (1/2*X.shape[0])*(error.T @ error)
        past_cost.append(loss)
        grad =(1/X.shape[0])*(X.T @(loss))
        w -= lr*grad
        past_w.append(w)
    plt.plot(past_cost)
    return w,y_pred

In [ ]:
#linear_reg(X_train,y_train,100,0.1)

In [ ]:
alpha = 0.01 #Step size
iterations = 10 #No. of iterations
m = y_train.size #No. of data points
np.random.seed(123) #Set the seed
theta = np.random.rand(X_train.shape[1],y_train.shape[0]) #Pick some random values to start with


#GRADIENT DESCENT
def gradient_descent(x, y, theta, iterations, alpha):
    past_costs = []
    past_thetas = [theta]
    for i in range(iterations):
        prediction = np.dot(x, theta)
        error = prediction - y
        cost = 1/(2*m) * np.dot(error.T, error)
        past_costs.append(cost)
        theta = theta - (alpha * (1/m) * np.dot(x.T, error))
        past_thetas.append(theta)
        
    return past_thetas, past_costs

#Pass the relevant variables to the function and get the new values back...
#past_thetas, past_costs = gradient_descent(X_train, y_train, theta, iterations, alpha)
#theta = past_thetas[-1]

#Print the results...
#print("Gradient Descent: {:.2f}, {:.2f}".format(theta[0], theta[1]))
